In [1]:
!pip install -q tensorflow tensorflow-datasets

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical


In [3]:
from sklearn.datasets import load_iris

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encode the labels
y = to_categorical(y, num_classes=3)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
model = Sequential([
    Dense(16, activation='relu', input_shape=(4,)),
    Dense(8, activation='relu'),
    Dense(3, activation='softmax')  # Output layer for 3 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.6912 - loss: 0.9291 - val_accuracy: 0.6333 - val_loss: 0.9190
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6521 - loss: 0.9034 - val_accuracy: 0.6333 - val_loss: 0.8603
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6684 - loss: 0.8478 - val_accuracy: 0.6667 - val_loss: 0.8017
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7089 - loss: 0.7906 - val_accuracy: 0.9333 - val_loss: 0.7473
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7610 - loss: 0.7303 - val_accuracy: 0.9000 - val_loss: 0.6899
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8063 - loss: 0.7025 - val_accuracy: 0.8667 - val_loss: 0.6330
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7783 - loss: 0.6445 - val_accuracy: 0.8333 - val_loss: 0.5808
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7553 - loss: 0.6178 - val_accuracy: 0.8333 - val_loss: 0.

In [5]:
# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('iris_model.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model converted to TensorFlow Lite format!")


Saved artifact at '/tmp/tmpzjhwvocf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140528577685536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140528333841760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140528333841408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140528333838064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140528333834016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140528333838768: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model converted to TensorFlow Lite format!


In [6]:
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='iris_model.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)


Input details: [{'name': 'serving_default_keras_tensor:0', 'index': 0, 'shape': array([1, 4], dtype=int32), 'shape_signature': array([-1,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 10, 'shape': array([1, 3], dtype=int32), 'shape_signature': array([-1,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [7]:
# Example input data (scaled similarly to training)
sample_input = np.array([[5.1, 3.5, 1.4, 0.2]])  # Replace with actual data
sample_input = scaler.transform(sample_input)  # Scale the input using the trained scaler
sample_input = np.array(sample_input, dtype=np.float32)  # Convert to float32

print("Prepared input:", sample_input)


Prepared input: [[-0.9006812  1.0190043 -1.3402265 -1.3154444]]


In [8]:
# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], sample_input)

# Run inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Inference output:", output_data)

# Decode the output
predicted_class = np.argmax(output_data)
print("Predicted class:", predicted_class)


Inference output: [[9.9890661e-01 1.0923198e-03 1.0652049e-06]]
Predicted class: 0
